# 2. Semantic Kernel の Planner を利用する

**Step 1**: Semantic Kernel の初期設定

In [ ]:
// Semantic Kernel パッケージのインポート、Kernel の初期化、Azure OpenAI Service の設定
#r "nuget: Microsoft.SemanticKernel, 0.15.230531.5-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using System.Text.Encodings.Web;
using System.Text.Json;

var builder = new KernelBuilder();
builder.WithAzureTextCompletionService("text-davinci-003", "https://YOUR_SERVICE_NAME.openai.azure.com/", "YOUR_API_KEY");
IKernel kernel = builder.Build();

**Step 2**: Planner の準備

In [ ]:
// Planner の初期化
var planner = new SequentialPlanner(kernel);

In [ ]:
// Skills ディレクトリから RecipeCreatorSkill をインポートする
var skillsDirectory
    = System.IO.Path.Combine(
        System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");
var recipeCreatorFunctions
    = kernel.ImportSemanticSkillFromDirectory(
        skillsDirectory, "RecipeCreatorSkill");

**Step 3**: Planner によるプラン作成と確認

In [ ]:
var ask = "今日はきつねうどんを食べたいです。足りない栄養素を補う献立メニューを作ってください";
var plan = await planner.CreatePlanAsync(ask);

In [ ]:
// プランで使用する Skill の表示
var planSteps = 
    string.Join(
        "\n", 
        plan.Steps.Select((step, i) => $"-step {i}: {step.Name}")
    );

Console.WriteLine(plan.Steps.Count + $" steps in plan:");
Console.WriteLine(planSteps);

In [ ]:
// プラン詳細の表示
var planJson = JsonSerializer.Serialize(
    plan, 
    new JsonSerializerOptions { 
        Encoder = JavaScriptEncoder.Create(System.Text.Unicode.UnicodeRanges.All),
        WriteIndented = true
        }
    );
Console.WriteLine(planJson);

**Step 4**: プランの実行

In [ ]:
var result = await plan.InvokeAsync();
Console.WriteLine(result);